<a href="https://colab.research.google.com/github/akib1162100/ML_base/blob/main/Bloom_7b_Casual_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
import warnings
from transformers import BloomModel, BloomTokenizerFast, BloomForCausalLM, BloomConfig, TextDataset, TrainingArguments, Trainer, pipeline, DataCollatorForLanguageModeling
warnings.filterwarnings('ignore')
import transformers, accelerate, peft
import os

bin c:\users\acer\downloads\llama or bloom\bloom\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [ ]:
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.31.0
Accelerate version: 0.21.0
PEFT version: 0.4.0


In [ ]:
dataset = load_dataset("text", data_files = "test3.txt", split = 'train')

Found cached dataset text (C:/Users/ACER/.cache/huggingface/datasets/text/default-da1166b43c04c884/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 39
})

In [ ]:
dataset['text'][5]

"আমার ওপর ভরসা রাখুন, আমি জেমস বন্ডকে এত সহজে এই ছবিতে উতরে যেতে দেব না।'"

In [ ]:
dataset = dataset.shuffle(seed = 42)

Loading cached shuffled indices for dataset at C:\Users\ACER\.cache\huggingface\datasets\text\default-da1166b43c04c884\0.0.0\cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2\cache-96c8a216dc97cf17.arrow


In [ ]:
dataset['text'][5]

"আন্তর্জাতিক দলও এ বিষয়টি নিশ্চিত করেছে।'"

In [ ]:
len(dataset['text'])

39

In [ ]:
# news= open('en_US/en_US.news.txt',encoding="utf8").read()
# # blogs= open('en_US/en_US.blogs.txt',encoding="utf8").read()
# # twitter= open('en_US/en_US.twitter.txt',encoding="utf8").read()

In [ ]:
def extra_space(text):
    new_text= re.sub("\s+"," ",text)
    return new_text
def sp_charac(text):
    new_text=re.sub("[^0-9A-Za-z ]", "" , text)
    return new_text

### Removing extra space and special characters to find the average word count in each sentence

In [ ]:
dataset['text'][1]

"অরুণাচলের কংগ্রেস দলীয় এমপি নিনং ইরিং মোদিকে পাঠানো চিঠিতে লিখেছেন, 'প্রমত্তা সিয়াং (ব্রহ্মপুত্র নদের আঞ্চলিক নাম) নদীর পানি এই নভেম্বর মাসে নোংরা ও কালো হওয়ার অন্য কোনো কারণ নেই।"

In [ ]:
for i in range(len(dataset['text'])) :
    dataset['text'][i] = extra_space(dataset['text'][i])
    # text_corpus[i] = sp_charac(text_corpus[i])

In [ ]:
dataset['text'][0]

''

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 39
})

### Dividing the sentences in train and test data and saving it in file

In [ ]:
dev = dataset.train_test_split(test_size=0.2)

In [ ]:
dev['train']['text']

["বন্ড সিরিজের নতুন পরিচালক ফুকুনাগা এর আগে এইচবিও চ্যানেলের জনপ্রিয় আমেরিকান টিভি সিরিজ 'ট্রু ডিটেকটিভ'-এর পরিচালক হিসেবে খ্যাতি অর্জন করেছেন।",
 "তখন ভয়ংকর নতুন প্রযুক্তি নিয়ে সদলবলে হাজির হবে এক রহস্যময় ভিলেন।'",
 'ব্রহ্মপুত্রের গতিপথ বদলাতে সুড়ঙ্গ নির্মাণ করছে চীন',
 'কিন্তু আমি শিগগিরই নতুন বন্ড সিরিজের শিল্পী আর কলাকুশলীদের সঙ্গে যোগ দেব।',
 'কোচিং সেন্টারে ওই হামলায় আফগান কর্তৃপক্ষ বৃহস্পতিবার ৩৪ জন নিহত এবং ৫৬ জন আহত হওয়ার কথা জানায়।',
 "সবকিছু ঠিক থাকলে ২০২০ সালের এপ্রিলে মুক্তি পাবে 'বন্ড টুয়েন্টি ফাইভ'। ডেকান ক্রনিকল",
 "নিনং ইরিং আরো লিখেছেন, 'অতীতে কখনো নদের পানির মান এতো খারাপ হওয়ার তথ্য স্থানীয় লোকজনেরও জানা নেই।",
 'তবে এর সপক্ষে তারা কোনো প্রমাণ দেয়নি।',
 "আফগান কোচিং সেন্টারে হামলার দায় স্বীকার আইএস'র",
 'বন্ড জ্যামাইকাতে অবকাশ কাটাচ্ছে।',
 'এটি বিশ্বে কৃত্রিমভাবে নদীর গতিপ্রবাহ পরিবর্তনের সবচেয়ে বড় পরিকল্পনা।',
 'এমন সময় সিআইএর পুরোনো বন্ধু ফেলিক্স লেইটার তাঁর কাছে সাহায্য চান।',
 "বন্ড সিরিজের নতুন ছবিতে দর্শক এমন কিছু আশা করে, যা তারা আগে দেখেনি।'",

In [ ]:
len(dev['train']['text'])

31

In [ ]:
len(dev['test']['text'])

8

In [ ]:
dev['test']

Dataset({
    features: ['text'],
    num_rows: 8
})

In [ ]:
import torch
from transformers import BloomForCausalLM, AutoTokenizer
model_checkpoint = "bigscience/bloom-7b1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
print('vocabulary size: %d, max sequence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))

vocabulary size: 250680, max sequence length: 1000000000000000019884624838656


In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

In [ ]:
tokenized_dt = dev.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=["text"],

)

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
tokenized_dt

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 31
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8
    })
})

In [ ]:
block_size = 256


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dt.map(group_texts, batched=True, num_proc=1)
lm_dataset

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

In [ ]:
lm_dataset['train']['labels'][2]

[711,
 281,
 127,
 2105,
 250,
 1771,
 2340,
 18398,
 720,
 1529,
 219,
 250,
 991,
 638,
 1529,
 219,
 711,
 281,
 126,
 2105,
 1529,
 219,
 250,
 1781,
 1529,
 213,
 2105,
 1529,
 213,
 1103,
 1529,
 219,
 250,
 3571,
 281,
 127,
 2105,
 1529,
 219,
 250,
 1726,
 281,
 126,
 2639,
 1726,
 281,
 124,
 281,
 126,
 250,
 1017,
 711,
 1529,
 219,
 640,
 790,
 281,
 126,
 281,
 214,
 2340,
 281,
 126,
 1093,
 1529,
 219,
 1781,
 233429,
 2004,
 281,
 126,
 2105,
 1103,
 250,
 2639,
 250,
 1901,
 1529,
 212,
 1009,
 1529,
 219,
 2105,
 250,
 1200,
 2004,
 281,
 127,
 1009,
 18398,
 1009,
 250,
 1009,
 1093,
 1529,
 212,
 250,
 711,
 18398,
 2105,
 1017,
 18398,
 965,
 720,
 1529,
 213,
 1103,
 18398,
 2105,
 1529,
 219,
 2105,
 250,
 1771,
 1103,
 281,
 127,
 720,
 3027,
 250,
 711,
 1093,
 2340,
 281,
 126,
 1103,
 1529,
 219,
 250,
 1901,
 1529,
 212,
 1009,
 250,
 1017,
 281,
 126,
 1453,
 281,
 126,
 2105,
 250,
 638,
 281,
 127,
 2340,
 1529,
 223,
 965,
 281,
 127,
 4541,
 281,
 126,

In [ ]:
tokenizer.decode(lm_dataset["train"][0]["input_ids"])

"ত খ ন   ভ য ় ং ক র   ন ত ু ন   প ্ র য ু ক ্ ত ি   ন ি য ় ে   স দ ল ব ল ে   হ া জ ি র   হ ব ে   এ ক   র হ স ্ য ম য ়   ভ ি ল ে ন । 'ত া র া   ব ি শ ্ ব ব ি দ ্ য া ল য ় ে   ভ র ্ ত ি   প র ী ক ্ ষ া য ়   অ ং শ   ন ে ও য ় া র   প ্ র স ্ ত ু ত ি   ন ি ত ে   ম া ব ৌ দ   এ ক া ড ে ম ি   ন া ম ে র   ও ই   ক ো চ ি ং   স ে ন ্ ট া র ে   ক ্ ল া স   ক র ছ ি ল ।এ র   ফ ল ে   ভ া র ত ে র   অ র ু ণ া চ"

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

In [ ]:
print(model.num_parameters())
model

559214592


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [ ]:
print_model_size(model)
print_trainable_parameters(model)

In [ ]:
from peft import LoraConfig, get_peft_model

target_modules = ["word_embeddings","h.query_key_value", "h.dense" "lm_head"]
config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules= target_modules,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
peft_model = get_peft_model(model, config)
print_model_size(peft_model)
print_trainable_parameters(peft_model)

In [ ]:
model.config.apply_spec_augment = True
# model.config.max_length = generation_max_length
model.config.dropout = 0.1
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
del model

In [ ]:
training_args = TrainingArguments(
    output_dir=".\Bloom_2\model_peft",
    evaluation_strategy="epoch",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 2,
    overwrite_output_dir = True,
    learning_rate=1e-3,
    save_strategy = 'epoch',
    weight_decay=0.001,
    push_to_hub=False,
    save_total_limit = 10,
    warmup_steps = 10,
    dataloader_num_workers = 0,
    logging_steps = 5,
    load_best_model_at_end = True,
#     metric_for_best_model = "perplexity",
    greater_is_better = False ,
    lr_scheduler_type="cosine",

#     logging_steps=10,
#     bf16 = True ,
    # bf16 = False
#     tf32 = True,
    num_train_epochs = 5,
    disable_tqdm = True,
    report_to = ["tensorboard"]
)

In [ ]:
import math
def compute_metrics(pred):
    return math.exp(pred['train'])

In [ ]:
# Initializing the trainer class object that will do the training
# here the data collator will generate the batch of size 64 of train and test data
trainer = Trainer(
    model = peft_model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = lm_dataset['train'],
    eval_dataset = lm_dataset['test'],
#     compute_metrics=compute_metrics,

)

In [ ]:
trainer.train()

{'loss': 3.1462, 'learning_rate': 0.0005, 'epoch': 0.83}
{'eval_loss': 3.070661783218384, 'eval_runtime': 0.5879, 'eval_samples_per_second': 5.103, 'eval_steps_per_second': 5.103, 'epoch': 1.0}
{'loss': 3.0238, 'learning_rate': 0.001, 'epoch': 1.67}
{'eval_loss': 3.004868268966675, 'eval_runtime': 0.5917, 'eval_samples_per_second': 5.071, 'eval_steps_per_second': 5.071, 'epoch': 2.0}
{'loss': 2.9223, 'learning_rate': 0.0008535533905932737, 'epoch': 2.5}
{'eval_loss': 2.933964967727661, 'eval_runtime': 0.6023, 'eval_samples_per_second': 4.981, 'eval_steps_per_second': 4.981, 'epoch': 3.0}
{'loss': 2.8454, 'learning_rate': 0.0005, 'epoch': 3.33}
{'eval_loss': 2.8998947143554688, 'eval_runtime': 0.603, 'eval_samples_per_second': 4.975, 'eval_steps_per_second': 4.975, 'epoch': 4.0}
{'loss': 2.7412, 'learning_rate': 0.00014644660940672628, 'epoch': 4.17}
{'loss': 2.7407, 'learning_rate': 0.0, 'epoch': 5.0}
{'eval_loss': 2.892390012741089, 'eval_runtime': 0.6032, 'eval_samples_per_second': 4

TrainOutput(global_step=30, training_loss=2.9032703399658204, metrics={'train_runtime': 35.8862, 'train_samples_per_second': 1.672, 'train_steps_per_second': 0.836, 'train_loss': 2.9032703399658204, 'epoch': 5.0})

In [ ]:
#2 epochs
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

{'eval_loss': 2.892390012741089, 'eval_runtime': 0.8499, 'eval_samples_per_second': 3.53, 'eval_steps_per_second': 3.53, 'epoch': 5.0}
Perplexity: 18.04


In [ ]:
#2 epochs Perplexity: 24.30
#3 epochs Perplexity: 23.53
#5 epochs Perplexity: 22.68


#### The train loss achieved is 4.778

In [ ]:
# Saving the model
trainer.save_model()

In [ ]:
# Evaluating on Test data
trainer.evaluate(lm_dataset['test'])

{'eval_loss': 2.892390012741089, 'eval_runtime': 0.9009, 'eval_samples_per_second': 3.33, 'eval_steps_per_second': 3.33, 'epoch': 5.0}


{'eval_loss': 2.892390012741089,
 'eval_runtime': 0.9009,
 'eval_samples_per_second': 3.33,
 'eval_steps_per_second': 3.33,
 'epoch': 5.0}

#### Test loss is 5.52

### Creating a pipeline object for text generation

In [ ]:
tokenizer.save_pretrained("Tokenizer_bloom1")

('Tokenizer_bloom1\\tokenizer_config.json',
 'Tokenizer_bloom1\\special_tokens_map.json',
 'Tokenizer_bloom1\\tokenizer.json')

In [ ]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained(".\\Bloom_2\\model_peft\\checkpoint-30")
tokenizer = AutoTokenizer.from_pretrained(".\\Tokenizer_bloom1")
model = BloomForCausalLM.from_pretrained(config.base_model_name_or_path)

In [ ]:
model.num_parameters

<bound method ModuleUtilsMixin.num_parameters of BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((10

In [ ]:
generator = pipeline('text-generation', tokenizer=tokenizer, model=model)

In [ ]:
'''
generating  next word in 3 possible ways
1. Greedy Search : chooses the best possible next word based on highest probability from 1 hypothesis
2. Beam Search : chooses the high probability next word from n hypothesis
3. Random Sampling : chooses random next word from possible hypothesis , however as the temperature is set high , it will
   ignore low probability words.
'''

print(generator('আমি বাংলায় গান', max_length=5)[0]['generated_text'])
print(generator('আমার নাম', max_length=5,num_beams = 5)[0]['generated_text'])
print(generator('তোমার কি' , max_length=5 , do_sample=True,temperature = 0.7)[0]['generated_text'])


আমি বাংলায় গান গাই


### Predicting Next word

In [ ]:
def predict_next():
    from transformers import OpenAIGPTTokenizer,OpenAIGPTLMHeadModel,\
    TextDataset,TrainingArguments,Trainer,pipeline,DataCollatorForLanguageModeling
    import re
    from nltk.tokenize import word_tokenize


    tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
    model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
    generator = pipeline('text-generation', tokenizer='openai-gpt', model='gpt_model')
    while(True):
        text = input('Enter the text: ')
        length= len(tokenizer.encode(text, return_tensors='pt')[0])

        max_length = length+1

        print('Next Word: ')
        print(generator(text , max_length=max_length)[0]['generated_text'].split(' ')[-1])
        print(generator(text , max_length=max_length , num_beams = 5)[0]['generated_text'].split(' ')[-1])
        print(generator(text , max_length=max_length , do_sample=True,temperature = 0.7)[0]['generated_text'].split(' ')[-1])




In [ ]:
predict_next()

In [ ]:
bloom stgcn whisper/wave2vec